In [48]:
%load_ext autoreload
%autoreload 2
%matplotlib qt5
# Add modules to the path
import sys
sys.path.insert(0, 'C:\\Users\\francescag\\Documents\\SourceTree_repos\\Python_git')
sys.path.insert(0, 'C:\\Users\\francescag\\Documents\\SourceTree_repos')

import joypy
import copy
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pickle
from utils.plotting import HeatMapParams
from utils.plotting import heat_map_and_mean
from utils.plotting import get_photometry_around_event
from utils.plotting import correctData, cueData, rewardData

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [65]:
mouse = 'SNL_photo17'
date = '20200204'
saving_folder = 'W:\\photometry_2AC\\processed_data\\' + mouse + '\\'
restructured_data_filename = mouse + '_' + date + '_' + 'restructured_data.pkl'
trial_data = pd.read_pickle(saving_folder + restructured_data_filename) 

In [40]:
trial_data['Max times in state'].unique()


array([1., 5., 2., 3., 4., 6., 8.])

In [53]:
demod_trace_filename = mouse + '_' + date + '_' + 'not_regress.npy'
signal = np.load(saving_folder + demod_trace_filename)

In [66]:
dff_trace_filename = mouse + '_' + date + '_' + 'smoothed_signal.npy'
dff = np.load(saving_folder + dff_trace_filename)

In [83]:
background_trace_filename = mouse + '_' + date + '_' + 'background.npy'
background = np.load(saving_folder + background_trace_filename)

In [29]:
#plt.plot(background)
#plt.plot(signal)
#plt.plot(dff)

In [44]:
state_type_of_interest = 3
response = 0
outcome = 3
last_outcome = 0 # NOT USED CURRENLY
no_repeats = 0
last_response = 0
align_to = 'Time start'
instance = 1
plot_range = [-6,6]
first_choice_correct = 0
first_choice = 2


plotting_params = HeatMapParams(state_type_of_interest, response, first_choice, last_response, outcome, last_outcome, first_choice_correct, align_to, instance, no_repeats, plot_range)
dff_events = heat_map_and_mean(trial_data,dff, plotting_params, mouse, date)

(113, 160000)
(113,) (113,)


Bootstrapping...: 100%|████████████████████| 1000/1000 [01:24<00:00, 11.87it/s]


7.134917699115028 4.502188188713288
0.09999999999990905


In [67]:
correct_data = correctData('left', mouse, date, trial_data, dff)
mean_and_sem_filename = mouse + '_' + date + '_' + 'mean_correct_data.p'
save_filename = saving_folder + mean_and_sem_filename
pickle.dump(correct_data,open( save_filename, "wb" ) )

(77, 160000)
(77,) (77,)


Bootstrapping...: 100%|████████████████████| 1000/1000 [00:58<00:00, 17.00it/s]


7.936649350649318 11.079484360365813
0.012900000000627188
(81, 160000)
(81,) (81,)


Bootstrapping...: 100%|████████████████████| 1000/1000 [01:02<00:00, 15.99it/s]


15.539203703703707 68.88915070303196
1.075699999999415


In [36]:
cue_data = cueData('left', mouse, date, trial_data, dff)
mean_and_sem_filename = mouse + '_' + date + '_' + 'mean_cue_data.p'
save_filename = saving_folder + mean_and_sem_filename
pickle.dump(cue_data,open( save_filename, "wb" ) )

(146, 160000)
(146,) (146,)


Bootstrapping...: 100%|████████████████████| 1000/1000 [01:49<00:00,  9.10it/s]


2.8068424657534212 2.507446929001357
0.09999999999990905
(146, 160000)
(146,) (146,)


Bootstrapping...: 100%|████████████████████| 1000/1000 [01:49<00:00,  9.11it/s]


2.627113698630145 0.7545280616648672
0.10000000000013642


In [37]:
reward_data = rewardData('left', mouse, date, trial_data, dff)
mean_and_sem_filename = mouse + '_' + date + '_' + 'mean_reward_data.p'
save_filename = saving_folder + mean_and_sem_filename
pickle.dump(reward_data,open( save_filename, "wb" ) )

(190, 160000)
(190,) (190,)


Bootstrapping...: 100%|████████████████████| 1000/1000 [02:20<00:00,  7.12it/s]


2.3856294736842 3.2059581022272545
-0.8280999999999494
(214, 160000)
(214,) (214,)


Bootstrapping...: 100%|████████████████████| 1000/1000 [02:39<00:00,  6.30it/s]


2.023806074766364 0.6053013089274248
-0.5529000000001361


In [20]:
state_type_of_interest = 5
state_type_of_interest1 = 8
state_type_of_interest2 = 10
response1 = 1
response2 = 2
outcome1 = 1
outcome2 = 0
last_outcome = 0 # NOT USED CURRENLY
no_repeats = 1
last_response = 0
align_to = 'Time end'
instance = -1
plot_range = [-3, 3]
first_choice_correct1 = 1
first_choice_correct2 = 0


plotting_params1 = HeatMapParams(state_type_of_interest, response1, first_choice, last_response, outcome1, last_outcome, first_choice_correct1, align_to, instance, no_repeats, plot_range)
plotting_params2 = HeatMapParams(state_type_of_interest, response2, first_choice, last_response, outcome2, last_outcome, first_choice_correct2, align_to, instance, no_repeats, plot_range)
#plotting_params1 = HeatMapParams(state_type_of_interest, response1, last_response, outcome1, last_outcome,first_choice_correct1, align_to, instance, no_repeats, plot_range)
#plotting_params2 = HeatMapParams(state_type_of_interest, response2, last_response, outcome1, last_outcome,first_choice_correct1,  align_to, instance, no_repeats, plot_range)
#plotting_params3 = HeatMapParams(state_type_of_interest, response1, last_response, outcome2, last_outcome,first_choice_correct2,  align_to, instance, no_repeats, plot_range)
#plotting_params4 = HeatMapParams(state_type_of_interest, response2, last_response, outcome2, last_outcome,first_choice_correct2,  align_to, instance, no_repeats, plot_range)
#plotting_params5 = HeatMapParams(state_type_of_interest1, response1, last_response, outcome1, last_outcome,first_choice_correct,  align_to, instance, no_repeats, plot_range)
#plotting_params6 = HeatMapParams(state_type_of_interest1, response2, last_response, outcome1, last_outcome,first_choice_correct,  align_to, instance, no_repeats, plot_range)
#plotting_params7 = HeatMapParams(state_type_of_interest2, response1, last_response, outcome1, last_outcome,first_choice_correct,  align_to, instance, no_repeats, plot_range)
#plotting_params8 = HeatMapParams(state_type_of_interest2, response2, last_response, outcome1, last_outcome,first_choice_correct,  align_to, instance, no_repeats, plot_range)
#events = multiple_conditions_plot(trial_data, dff, mouse, date, plotting_params1, plotting_params2, plotting_params3, plotting_params4)
events = multiple_conditions_plot(trial_data, dff, mouse, date, plotting_params1, plotting_params2)
#events = multiple_conditions_plot(trial_data, dff, mouse, date, plotting_params5, plotting_params6, plotting_params7, plotting_params8)


NameError: name 'first_choice' is not defined